Script serves two purposes:
Curates data for 3d grader
Builds models for grading EiPE answers (Data is 3D)

In [280]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.mixture import GaussianMixture
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold


from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier


import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import ConfusionMatrixDisplay


nltk.download('punkt')
nltk.download('words')

from nltk.corpus import words




[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\chine\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\chine\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [281]:
data_file = "sigcse_2024.csv"
alldata_df = pd.read_csv(data_file, encoding="utf-8")


In [282]:
alldata_df.head()

,subset,embedding_subset,embedding_index,pl_qid,qid,code,assumption,example_correct_answers,response,binary_annotators,...,chinny_una,chinny_c,chinny_hl,binglin_una,binglin_c,binglin_hl,una,c,hl,3d_resolution
0,train,train,97,cdrd_exam1_manual/cdrd_exam1_1,print_found_if_three_numbers_unique,"def f(x, y, z):\n if x != y and x != z and ...","Assume that the variables x, y, and z are inte...",['Print the word found if all three numbers ar...,the code checks if all arguments regarding var...,"['Austin', 'Max']",...,0,0,0.0,0,0,0,1,0,0,reconcile
1,validate,train,113,cdrd_exam1_manual/cdrd_exam1_1,print_items_out_smaller_then_larger,"def f(x, y):\n if x < y:\n print(x, ...",Assume that the variables x and y are integers.,['prints two given numbers in numberical order...,prints two numbers ordered smallest to largest,"['Austin', 'Max']",...,1,1,1.0,1,1,1,0,1,1,reconcile
2,train,train,138,cdrd_exam1_manual/cdrd_exam1_2,computes_average_of_list,def f(x):\n return sum(x) / len(x),Assume that the variable x is a non-empty list...,"['Returns the average value in a list', 'finds...",returns the average of a list,"['Binglin', 'Rachel']",...,1,1,1.0,1,1,1,1,1,1,reconcile
3,validate,train,150,cdrd_week5_exam,does_list_have_value_above_y,"def f(x, y):\n for val in x:\n if va...",Assume that the variable x is a list of intege...,['Return True if at least one list element is ...,returns whether the first value in a list is g...,['Binglin'],...,1,0,1.0,1,0,1,1,0,1,reconcile
4,train,train,228,cdrd_exam3_manual/cdrd_exam3_1,count how many times a given string appears in...,"def f(x, y):\n k = 0\n for e in x:\n ...",Assume that variable x is a list of strings an...,['Return a count of how many times a given str...,return the number of times in which the string...,"['Binglin', 'Rachel']",...,1,1,0.0,1,1,1,1,1,1,reconcile


In [283]:
print(alldata_df.shape)

(3064, 24)


In [284]:
def display_class_dist(given_df):
    '''
    Display the distribution of data for each qid. There are 8 possible bins.
    '''
    classes =  {(0,0,0): 0, (0,0,1):0, (0,1,0):0, (0,1,1):0, 
                (1,0,0):0, (1,1,0):0, (1,0,1):0, (1,1,1):0}
    
    ordering = [(0,0,0), (0,0,1), (0,1,0), (0,1,1), (1,0,0), (1,1,0), (1,0,1), (1,1,1)]
    
    print("3D groupings = (unambig, correct, highlevel)\n\n")
    print("3D label,  Freq. (Num Times Occur)")
    
    numrowsdata = 0
    
    for index, row in given_df.iterrows():
        
        cur_unambig = row["una"]
        cur_correct = row["c"]
        cur_hl      = row["hl"]
        
        classes[(cur_unambig, cur_correct, cur_hl)] += 1
        numrowsdata += 1
    
    for curbin in ordering:
        curbincount = classes[curbin] #get the number of occurrences of a particular labeling configuration within the data
        print("{}  {:>5.2f}% ({})".format(curbin, round(100*curbincount/numrowsdata, 2), curbincount)) #print group name, group freq and raw count of group occur.


    
def qid_dist(given_df):
    """
    Show how frequently each qid occured in the data
    """
    pass

In [285]:
filter1 = alldata_df[alldata_df["subset"] == "train"]
print(filter1.shape)
filter2= filter1[filter1["qid"] == "computes_average_of_list"]
print(filter2.shape)
display_class_dist(filter2)

display_class_dist(alldata_df[alldata_df["qid"] == "computes_average_of_list"])

(1832, 24)
(72, 24)
3D groupings = (unambig, correct, highlevel)


3D label,  Freq. (Num Times Occur)
(0, 0, 0)   0.00% (0)
(0, 0, 1)   0.00% (0)
(0, 1, 0)   0.00% (0)
(0, 1, 1)   8.33% (6)
(1, 0, 0)   0.00% (0)
(1, 1, 0)  44.44% (32)
(1, 0, 1)   0.00% (0)
(1, 1, 1)  47.22% (34)
3D groupings = (unambig, correct, highlevel)


3D label,  Freq. (Num Times Occur)
(0, 0, 0)   0.00% (0)
(0, 0, 1)   0.00% (0)
(0, 1, 0)   0.00% (0)
(0, 1, 1)   9.17% (11)
(1, 0, 0)   0.83% (1)
(1, 1, 0)  42.50% (51)
(1, 0, 1)   0.00% (0)
(1, 1, 1)  47.50% (57)


In [286]:
#distribution of all the data, even those tuples that were manually reconciled.
display_class_dist(alldata_df)

3D groupings = (unambig, correct, highlevel)


3D label,  Freq. (Num Times Occur)
(0, 0, 0)   6.01% (184)
(0, 0, 1)  13.38% (410)
(0, 1, 0)   1.44% (44)
(0, 1, 1)   9.76% (299)
(1, 0, 0)   4.44% (136)
(1, 1, 0)   6.59% (202)
(1, 0, 1)  20.30% (622)
(1, 1, 1)  38.09% (1167)


In [287]:
stemmer = PorterStemmer()

def preprocess_text(studentresponse):    
    words = word_tokenize(studentresponse)
    stemmed_words = [stemmer.stem(word) for word in words]
    lower_words= [word.lower() for word in stemmed_words] #convert to lowercase
    stemmed_words = lower_words
    regular_alpha = [] 
    for word in stemmed_words:
        if word.isalnum():  #remove any word that isn't an alphabet or a number.
            regular_alpha.append(word)
    stemmed_words = regular_alpha
    
    #TODO - need to handle stop words by passing in my own list. (related to tf-idf weighting)
    
    return " ".join(stemmed_words)

In [288]:
#distance metrics
#TODO: consider a few metrics


def cosine_similarity(x, y):
    #compute the cosine similarity of two vectors

    return sklearn.metrics.pairwise.cosine_similarity(x, y).item()     #using .item() to get value as a scalar
    
def distance_from_golden(golden, given_ans, dist_func):
    #compute a distance metric for the distance between a given answer and the golden answer
    distance = dist_func(golden, given_ans)
    if distance == None:
        return float("inf")
    else:
        return distance

In [289]:
#create the groupings (e.g 0-1-1 will be marked as 1 for ambiguous, any other combo is 0 for ambiguous)
def make_binary_col_if_group_found(givendf1, givendims, newcolname):
    
    tuple_found = []
    
    for index, row in givendf1.iterrows():
        
        cur_unambig = row["una"]
        cur_correct = row["c"]
        cur_hl      = row["hl"]
        
        cur_pair = (cur_unambig, cur_correct, cur_hl)
        if cur_pair == givendims:
            tuple_found.append(1)
        else:
            tuple_found.append(0)
    
    newdf = givendf1.copy()
    res = pd.Series(tuple_found)
    newdf[newcolname] = res
    
    #givendf[newcolname] = pd.Series(tuple_found)
    
    print("unique values in tuple = ", set(tuple_found))
    
    print("the numbers stored are ", res.to_string())
    return newdf

In [290]:
def plot_confusion_matrix(confusion_matrix, what_to_predict, accuracy ):
        '''
        Function to help with plotting a confusion matrix
        '''
        plt.figure(figsize=(9,9))
        sns.heatmap(confusion_matrix, annot=True, fmt=".3f", linewidths=.5, square = True, cmap = 'Blues_r');
        plt.ylabel('Actual label');
        plt.xlabel('Predicted label');
        all_sample_title = 'Accuracy Score for {thingtopredict}: {accscore:.2f}'.format(thingtopredict = what_to_predict,
                                                                                    accscore = accuracy)
        plt.title(all_sample_title, size = 15);
        

def process_raw_features(given_df, pct_for_train = 0.8):
    """
    Process the raw text and get it ready to be trained
    Return processed text as a list of lists, where each list is one student response or one row of data
    """
    
    numsamples = 0
    limit = int(pct_for_train * len(given_df["response"]))
    processed_studentanswers = []
    
    for studentanswer in given_df["response"]:
        
        processed_answer = preprocess_text(studentanswer)
        processed_studentanswers.append(processed_answer)
        numsamples += 1
        
        if numsamples == limit:
            break
    return processed_studentanswers


def add_similarity_vector(golden_answers, students_responses_as_vec, curvec_fitter):
    """
    Create a column vector that represents the distance of a given student answer from the golden answer
    """
    golden_answers_vectors = curvec_fitter.transform(golden_answers).toarray()
    
    sim_vector = []
    
    #print("Shape of all student vectors = {}".format(students_responses_as_vec.shape))
    #print("Shape of golden_answers_vectors = {}".format(golden_answers_vectors.shape))
    
    for curstudent_vec in students_responses_as_vec:
        
        #print("Shape single student vector = {}".format(curstudent_vec.shape))
        best_similar = float("-inf") #ranges from 0 to 1 (1 meaning very similar or identical)
        
        for single_golden_answer_vec in golden_answers_vectors:
            #print("Shape single golden answer vector = {}".format(single_golden_answer_vec.shape))
            #cur_distance = distance_from_golden(single_golden_answer_vec, curstudent_vec, cosine_similarity)
            #compute distance. reshape the data into row vectors
            
            parta = single_golden_answer_vec.reshape(1,-1)
            partb = curstudent_vec.reshape(1,-1)
            
            #print("After reshaping, current size = {} for golden answer and  {} for student vector".format(parta.shape, partb.shape))
            
            cur_distance = cosine_similarity(parta, partb)
            best_similar = max(best_similar, cur_distance)
            
        sim_vector.append(best_similar)
    
    sim_vector = np.array(sim_vector)
    sim_vector = sim_vector.reshape(-1,1) #need to convert into a proper column vector
    return sim_vector


def train_model_and_validate(given_train_df, given_validate_df, golden_answers = [], qname = ""):
    
    ydims =  ["una", "c", "hl"]
    
    
    X_train_processed_text = process_raw_features(given_train_df, pct_for_train = 1)
 
    vectorizer = CountVectorizer(ngram_range = (2,2), min_df = 2)
    vec_fitter = vectorizer.fit(X_train_processed_text)    #fit only on training data to prevent overfit
    
    
    X_train = vec_fitter.transform(X_train_processed_text).toarray() #transform data into vectors and convert it to array    
    y_train = given_train_df[ydims].to_numpy(dtype=int)
    
    X_validate_processed_text = process_raw_features(given_validate_df, pct_for_train = 1)
    X_validate = vec_fitter.transform(X_validate_processed_text).toarray()  #use the previously fitted vectorizer to maintain shape
    y_validate = given_validate_df[ydims].to_numpy(dtype=int)
    
    if len(golden_answers) > 0:
    
        distance_from_golden_vector_trainset = add_similarity_vector(golden_answers, X_train, vec_fitter)
        distance_from_golden_vector_validationset  = add_similarity_vector(golden_answers, X_validate, vec_fitter)
        
        print("Original X data shape = {}, golden_answer_shape = {}".format(X_train.shape,
                                                                           distance_from_golden_vector_trainset.shape))
        
        X_train = np.append(X_train, distance_from_golden_vector_trainset, axis = 1)
        X_validate = np.append(X_validate, distance_from_golden_vector_validationset, axis = 1)
        
#         X_train.append(distance_from_golden_vector_trainset, axis = 1)
#         X_validate.append(distance_from_golden_vector_validationset, axis = 1)
    
    
    
 
        
    labels_order = ["Unambig", "Correct", "High level"]
    accuracy_l = []
    
    

    for i in range(3):
        model = LogisticRegression(random_state = 0)
        
        num_classes = list(np.unique(y_train[:,i])) #check number of classes in data. should be two
        print("We have {} classes".format(len(num_classes)))
        if len(num_classes) == 1: #change the last y in the tuple to force training to go through - otherwise, throwing error
            y_train[-1, i] = 0 if y_train[-1, i] == 1 else 0
        
        model = model.fit(X_train, y_train[:,i]) #train only one column at a time
        
        binary_y_predictions = model.predict(X_validate)
        
        #use the withheld set for testing the model
        print("{} Log reg. model performance on the withheld test set:\n\n".format(labels_order[i]))
        
        #for the y_test data (the true labels for column i), check if the predictions match
        dim_accuracy = metrics.accuracy_score(y_validate[:, i], binary_y_predictions)
        print("Accuracy is = ", dim_accuracy)
        accuracy_l.append(dim_accuracy)
        #print(metrics.classification_report(y_test, binary_y_predictions))


        print("\n\nConfusion Matrix:")
        
        confusion_matrix = metrics.confusion_matrix(y_validate[:, i],binary_y_predictions, normalize="true")
        print(confusion_matrix)
        
       
        #uncomment next line to print the confusion matrix
        #plot_confusion_matrix(confusion_matrix, labels_order[i] + qname,  dim_accuracy)
        print("\n\n")
        
    return accuracy_l  #accuracy for the three dimensions for a given problem


In [291]:
#for each question, create the input representation
#train various models, bigrams, bigrams + distance_from_golden, report on performance

train_pct = 1 #num from 0 to 1. currently 1 because want to train with 100% of train dataset
qids_accuracy = []

col_wrong = ["not unambig", "not c", "not hl"]
col_for_feedback = [(0,1,1), (1,0,1), (1,1,0)]

#col_for_feedback = {"not hl": (1,1,0), "not c": (1,0,1), "not unambig": (0,1,1)}

#make sure that we have the one vs all for all data before splitting
for i in range(3):
    #alldata_df = make_binary_col_if_group_found(alldata_df,  col_for_feedback[col_wrong], col_wrong)
    alldata_df = make_binary_col_if_group_found(alldata_df,  col_for_feedback[i], col_wrong[i])
    
    
train_df = alldata_df[alldata_df["subset"] == "train"]       #select only training rows
validate_df = alldata_df[alldata_df["subset"] == "validate"] #only validation row
test_df = alldata_df[alldata_df["subset"] == "test"]


#alldata_df[alldata_df["subset"] == "train" & alldata_df["qid"] == "computes average of list"]


for cur_qid in alldata_df["qid"].unique().tolist():
    
    cur_question_training_df = train_df[train_df["qid"] == cur_qid]
    cur_question_validation_df = validate_df[validate_df["qid"] == cur_qid]
    
    print("QID = \'{}.\'".format(cur_qid))
    qname_formatted = ": " + cur_qid #format cur_qid name so it looks nice when printed with other function
    
    cur_qid_accuracy = train_model_and_validate(cur_question_training_df, cur_question_validation_df, qname = qname_formatted)
    
    qids_accuracy.append(cur_qid_accuracy)
    print("\n\n")


  

unique values in tuple =  {0, 1}
the numbers stored are  0       0
1       1
2       0
3       0
4       0
5       0
6       0
7       0
8       1
9       0
10      0
11      0
12      0
13      1
14      0
15      0
16      1
17      0
18      0
19      0
20      0
21      0
22      0
23      0
24      0
25      0
26      0
27      0
28      0
29      0
30      1
31      1
32      0
33      1
34      0
35      1
36      0
37      0
38      0
39      0
40      0
41      0
42      0
43      0
44      0
45      0
46      0
47      0
48      0
49      0
50      0
51      0
52      0
53      0
54      0
55      0
56      0
57      0
58      0
59      0
60      0
61      0
62      0
63      1
64      0
65      0
66      1
67      0
68      0
69      1
70      0
71      0
72      0
73      0
74      1
75      0
76      1
77      0
78      0
79      0
80      0
81      0
82      0
83      1
84      0
85      0
86      0
87      1
88      0
89      0
90      0
91      0
92      0
93      0
94 

unique values in tuple =  {0, 1}
the numbers stored are  0       0
1       0
2       0
3       1
4       0
5       0
6       0
7       0
8       0
9       0
10      1
11      0
12      0
13      0
14      0
15      1
16      0
17      0
18      0
19      0
20      0
21      0
22      0
23      1
24      0
25      0
26      0
27      0
28      1
29      0
30      0
31      0
32      0
33      0
34      0
35      0
36      0
37      0
38      0
39      1
40      1
41      0
42      0
43      0
44      1
45      0
46      1
47      0
48      1
49      1
50      0
51      0
52      0
53      0
54      0
55      0
56      0
57      0
58      0
59      0
60      0
61      0
62      1
63      0
64      1
65      0
66      0
67      0
68      0
69      0
70      0
71      0
72      0
73      0
74      0
75      0
76      0
77      0
78      0
79      0
80      0
81      1
82      1
83      0
84      0
85      0
86      0
87      0
88      0
89      0
90      0
91      0
92      0
93      0
94 

unique values in tuple =  {0, 1}
the numbers stored are  0       0
1       0
2       0
3       0
4       0
5       0
6       0
7       0
8       0
9       0
10      0
11      0
12      0
13      0
14      0
15      0
16      0
17      0
18      0
19      0
20      0
21      0
22      0
23      0
24      0
25      0
26      0
27      1
28      0
29      1
30      0
31      0
32      0
33      0
34      0
35      0
36      0
37      0
38      0
39      0
40      0
41      1
42      0
43      0
44      0
45      0
46      0
47      0
48      0
49      0
50      0
51      0
52      1
53      1
54      0
55      0
56      0
57      0
58      0
59      0
60      0
61      0
62      0
63      0
64      0
65      0
66      0
67      1
68      0
69      0
70      1
71      1
72      1
73      1
74      0
75      0
76      0
77      0
78      0
79      0
80      0
81      0
82      0
83      0
84      0
85      0
86      0
87      0
88      0
89      0
90      0
91      0
92      0
93      0
94 

We have 2 classes
Unambig Log reg. model performance on the withheld test set:


Accuracy is =  0.8144329896907216


Confusion Matrix:
[[0.42857143 0.57142857]
 [0.02898551 0.97101449]]



We have 2 classes
Correct Log reg. model performance on the withheld test set:


Accuracy is =  0.8144329896907216


Confusion Matrix:
[[0.53846154 0.46153846]
 [0.08450704 0.91549296]]



We have 2 classes
High level Log reg. model performance on the withheld test set:


Accuracy is =  0.8969072164948454


Confusion Matrix:
[[0.82142857 0.17857143]
 [0.07246377 0.92753623]]






QID = 'computes_average_of_list.'
We have 2 classes
Unambig Log reg. model performance on the withheld test set:


Accuracy is =  0.9166666666666666


Confusion Matrix:
[[0. 1.]
 [0. 1.]]



We have 1 classes
Correct Log reg. model performance on the withheld test set:


Accuracy is =  0.9583333333333334


Confusion Matrix:
[[0. 1.]
 [0. 1.]]



We have 2 classes
High level Log reg. model performance on the withheld test se

In [292]:
#2nd model which incorporates the distance to the closest golden answer 

import ast
        

for cur_qid in alldata_df["qid"].unique().tolist():
    
        
    cur_question_training_df = train_df[train_df["qid"] == cur_qid]
    cur_question_validation_df = validate_df[validate_df["qid"] == cur_qid]
    
    print("QID = \'{}.\'".format(cur_qid))
    qname_formatted = ": " + cur_qid #format cur_qid name so it looks nice when printed with other function
    
    q_goldenanswers = cur_question_training_df["example_correct_answers"].iloc[0]
    q_goldenanswers = ast.literal_eval(q_goldenanswers)
    
    if type(q_goldenanswers) != list:
        print("Had to convert to list")
        q_goldenanswers = [q_goldenanswers]

    
    cur_qid_accuracy = train_model_and_validate(cur_question_training_df, cur_question_test_df, 
                                                qname = qname_formatted, golden_answers = q_goldenanswers)
    
    qids_accuracy.append(cur_qid_accuracy)
    print("\n\n")


QID = 'print_found_if_three_numbers_unique.'
Original X data shape = (96, 116), golden_answer_shape = (96, 1)
We have 2 classes
Unambig Log reg. model performance on the withheld test set:


Accuracy is =  0.6428571428571429


Confusion Matrix:
[[0.2        0.8       ]
 [0.11111111 0.88888889]]



We have 2 classes
Correct Log reg. model performance on the withheld test set:


Accuracy is =  0.6428571428571429


Confusion Matrix:
[[1.         0.        ]
 [0.83333333 0.16666667]]



We have 2 classes
High level Log reg. model performance on the withheld test set:


Accuracy is =  0.9285714285714286


Confusion Matrix:
[[0. 1.]
 [0. 1.]]






QID = 'print_items_out_smaller_then_larger.'
Original X data shape = (291, 351), golden_answer_shape = (291, 1)
We have 2 classes
Unambig Log reg. model performance on the withheld test set:


Accuracy is =  0.42857142857142855


Confusion Matrix:
[[0.8        0.2       ]
 [0.77777778 0.22222222]]



We have 2 classes
Correct Log reg. model perfor

[[0.         1.        ]
 [0.07692308 0.92307692]]






QID = 'increase_all_numbers_by_y.'
Original X data shape = (64, 65), golden_answer_shape = (64, 1)
We have 2 classes
Unambig Log reg. model performance on the withheld test set:


Accuracy is =  0.5714285714285714


Confusion Matrix:
[[0.         1.        ]
 [0.11111111 0.88888889]]



We have 2 classes
Correct Log reg. model performance on the withheld test set:


Accuracy is =  0.5714285714285714


Confusion Matrix:
[[1. 0.]
 [1. 0.]]



We have 2 classes
High level Log reg. model performance on the withheld test set:


Accuracy is =  0.8571428571428571


Confusion Matrix:
[[0.         1.        ]
 [0.07692308 0.92307692]]






QID = 'absolute_value.'
Original X data shape = (45, 46), golden_answer_shape = (45, 1)
We have 2 classes
Unambig Log reg. model performance on the withheld test set:


Accuracy is =  0.6428571428571429


Confusion Matrix:
[[0.2        0.8       ]
 [0.11111111 0.88888889]]



We have 2 classes
Correct 

In [293]:
#show the accuracy of the classifier for each column across all data



qids_accuracy = np.array(qids_accuracy)
print("Averaging results for all qids in each of the respective dimensions\n")

avg_unambig_accuracy = np.mean(qids_accuracy[:, 0])
print("Avg accuracy for Unambig: {:.2f}".format(avg_unambig_accuracy))


avg_corr_accuracy = np.mean(qids_accuracy[:, 1])
print("Avg accuracy for Correct column: {:.2f}".format(avg_corr_accuracy))



avg_highlvl_accuracy = np.mean(qids_accuracy[:, 2])
print("Avg accuracy for High-level column: {:.2f}\n\n".format(avg_highlvl_accuracy))


avg_classifier_perf_overall = np.mean(qids_accuracy)
print("Average results across all dimensions for all qids: {:.2f}".format(avg_classifier_perf_overall))

Averaging results for all qids in each of the respective dimensions

Avg accuracy for Unambig: 0.62
Avg accuracy for Correct column: 0.69
Avg accuracy for High-level column: 0.85


Average results across all dimensions for all qids: 0.72


In [294]:
def train_model_and_validate_one_versus_all(given_train_df, given_validate_df, one_vs_all_col, golden_answers = [], qname = ""):
    
    ydims =  ["una", "c", "hl"]
    
    
    X_train_processed_text = process_raw_features(given_train_df, pct_for_train = 1)
 
    vectorizer = CountVectorizer(ngram_range = (2,2), min_df = 2)
    vec_fitter = vectorizer.fit(X_train_processed_text)    #fit only on training data to prevent overfit
    
    
    X_train = vec_fitter.transform(X_train_processed_text).toarray() #transform data into vectors and convert it to array    
    y_train = given_train_df[one_vs_all_col].to_numpy(dtype=int) #training on special column
    print("Raw Ys = \n\n", given_train_df[one_vs_all_col].to_string())
    print("------------------------")
    X_validate_processed_text = process_raw_features(given_validate_df, pct_for_train = 1)
    X_validate = vec_fitter.transform(X_validate_processed_text).toarray()  #use the previously fitted vectorizer to maintain shape
    y_validate = given_validate_df[one_vs_all_col].to_numpy(dtype=int)
    
    
    print("validating on {} col".format(one_vs_all_col))
    
    if len(golden_answers) > 0:
    
        distance_from_golden_vector_trainset = add_similarity_vector(golden_answers, X_train, vec_fitter)
        distance_from_golden_vector_validationset  = add_similarity_vector(golden_answers, X_validate, vec_fitter)
        
        print("Original X data shape = {}, golden_answer_shape = {}".format(X_train.shape,
                                                                           distance_from_golden_vector_trainset.shape))
        
        X_train = np.append(X_train, distance_from_golden_vector_trainset, axis = 1)
        X_validate = np.append(X_validate, distance_from_golden_vector_validationset, axis = 1)
    
    
    #ensure that we have at least one tuple matching the condition by perturbing the input. 
    num_classes = list(np.unique(y_train)) #check number of classes in data. should be two
    print("We have {} classes".format(len(num_classes)))
    print("Classes are = ", np.unique(y_train))
    if len(num_classes) == 1: #change the last y in the tuple to force training to go through - otherwise, throwing error
            
            #y_train[-1] = 0 if y_train[-1] == 1 else 0
            if y_train[-1] == 1:
                
                y_train[-1] = 0
            
            elif y_train[-1] == 0:
                
                y_train[-1] = 1
                
            else:
                raise ValueError("Invalid Entry")
                
            print("We now have {} classes in y train".format(len(np.unique(y_train))))
            
    model = LogisticRegression(random_state = 0)
    model = model.fit(X_train, y_train) #train only one column, in this case one of the one-vs-all classifiers
        
    binary_y_predictions = model.predict(X_validate) #validate on the original data
        
        
        #for the y_test data (the true labels for column i), check if the predictions match
    dim_accuracy = metrics.accuracy_score(y_validate, binary_y_predictions)
    print("Accuracy for {} is = {} ".format(one_vs_all_col, dim_accuracy))
    #print(metrics.classification_report(y_test, binary_y_predictions))


    print("\n\nConfusion Matrix:")    
    confusion_matrix = metrics.confusion_matrix(y_validate, binary_y_predictions, normalize="true")
    print(confusion_matrix)
        
       
    #uncomment next line to print the confusion matrix
    #plot_confusion_matrix(confusion_matrix, labels_order[i] + qname,  dim_accuracy)
    print("\n\n")
        
    return dim_accuracy  #accuracy for the three dimensions for a given problem


In [295]:
#Models for training 1-1-0, 1-0-1, 0-1-1



import ast

one_v_all = []

for cur_qid in alldata_df["qid"].unique().tolist():
    
        
    cur_question_training_df = train_df[train_df["qid"] == cur_qid].copy()
    cur_question_validation_df = validate_df[validate_df["qid"] == cur_qid].copy()
    cur_question_test_df = test_df[test_df["qid"] == cur_qid].copy()
    
    one_v_all_current = [] #store the accuracy numbers for a one-vs-all classifier
    
    print("QID = \'{}.\'".format(cur_qid))
    qname_formatted = ": " + cur_qid #format cur_qid name so it looks nice when printed with other function
    
    q_goldenanswers = cur_question_training_df["example_correct_answers"].iloc[0]
    q_goldenanswers = ast.literal_eval(q_goldenanswers)
    
    if type(q_goldenanswers) != list:
        print("Had to convert to list")
        q_goldenanswers = [q_goldenanswers]
    
    
    
    for i in range(3):
       # cur_question_training_df = make_binarycol_if_group_found(cur_question_training_df, col_for_feedback[col_wrong],  col_wrong)
       #cur_question_validation_df = make_binarycol_if_group_found(cur_question_validation_df, col_for_feedback[col_wrong], col_wrong)
        print("Right before training, Y = ", list(cur_question_training_df[col_wrong[i]]))
        cur_qid_accuracy_dim = train_model_and_validate_one_versus_all(cur_question_training_df, cur_question_test_df, 
                                                                       col_wrong[i],
                                                qname = qname_formatted, golden_answers = q_goldenanswers)
        
        one_v_all_current.append(cur_qid_accuracy_dim)
        
        
    one_v_all.append(one_v_all_current)
    print("\n\n")


QID = 'print_found_if_three_numbers_unique.'
Right before training, Y =  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
Raw Ys = 

 0       0
43      0
60      0
86      0
92      0
108     0
117     0
179     0
220     0
251     0
312     0
371     1
388     0
410     0
434     1
438     0
445     0
453     0
481     1
487     1
520     0
574     0
579     0
639     0
706     0
728     0
751     0
777     0
778     1
802     0
803     0
906     0
911     0
962     0
1009    0
1142    1
1155    0
1184    0
1198    0
1200    0
1225    0
1289    1
1312    0
1374    0
1376    0
1433    0
1468    0
1472    0
1512    0
1566    0
1636    0
1639    0
1683    0
1697    0
1767    1
1896    1
1909    0
1917    0
1997    0
2040    0
2083    0
2088    0
2114  

Original X data shape = (291, 351), golden_answer_shape = (291, 1)
We have 2 classes
Classes are =  [0 1]
Accuracy for not unambig is = 0.8469387755102041 


Confusion Matrix:
[[0.98795181 0.01204819]
 [0.93333333 0.06666667]]



Right before training, Y =  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 

Original X data shape = (291, 351), golden_answer_shape = (291, 1)
We have 2 classes
Classes are =  [0 1]
Accuracy for not hl is = 0.8979591836734694 


Confusion Matrix:
[[0.97674419 0.02325581]
 [0.66666667 0.33333333]]






QID = 'computes_average_of_list.'
Right before training, Y =  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Raw Ys = 

 2       0
27      0
52      0
84      0
143     0
169     0
222     0
224     0
239     0
266     0
302     0
336     0
360     0
401     0
414     0
425     0
479     0
504     1
527     0
575     0
614     0
635     0
644     0
717     0
731     1
765     0
785     0
811     0
853     0
895     0
907     0
925     0
960     0
1016    0
1053    0
1057    0
1059    0
1088    0
1104    1
1109    0
1174    1
1237    0
1240    1
1381    0
1425    0
1446    0
1545    1
1593    0
1

Raw Ys = 

 4       0
33      0
80      0
151     0
162     0
249     1
268     0
318     0
338     0
339     0
375     0
436     0
475     0
496     0
563     0
620     0
654     0
749     0
786     0
787     0
949     0
956     0
1065    0
1103    0
1144    0
1226    0
1259    0
1262    0
1320    0
1358    0
1373    0
1402    0
1416    0
1488    0
1527    0
1607    0
1622    0
1704    0
1727    0
1774    0
1852    0
1911    0
1922    0
1935    0
1968    0
1996    0
2094    0
2164    0
2218    0
2222    0
2259    0
2338    0
2372    0
2438    0
2442    0
2472    0
2525    0
2562    0
2568    0
2585    0
2662    0
2680    0
2686    0
2689    0
2715    0
2725    0
2769    0
2786    0
2802    0
2917    0
2931    0
------------------------
validating on not hl col
Original X data shape = (71, 90), golden_answer_shape = (71, 1)
We have 2 classes
Classes are =  [0 1]
Accuracy for not hl is = 0.9583333333333334 


Confusion Matrix:
[[1. 0.]
 [1. 0.]]






QID = 'count how many numbers in a 

Original X data shape = (161, 240), golden_answer_shape = (161, 1)
We have 2 classes
Classes are =  [0 1]
Accuracy for not c is = 0.8148148148148148 


Confusion Matrix:
[[1. 0.]
 [1. 0.]]



Right before training, Y =  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Raw Ys = 

 11      0
24      0
44      0
83      0
124     0
134     0
201     0
221     0
228     0
229     0
231     0
259     0
264     0
273     0
278     0
288     0
291     0
298     0
323     0
364     0
378     0
400     0
406     0
415     0
431     0
443     0
450     0
456     0
482 

Original X data shape = (189, 188), golden_answer_shape = (189, 1)
We have 2 classes
Classes are =  [0 1]
Accuracy for not unambig is = 0.9682539682539683 


Confusion Matrix:
[[1. 0.]
 [1. 0.]]



Right before training, Y =  [1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1]
Raw Ys = 

 46      1
64      1
94      1
145     0
165     0
174     0
198     0
211     0
212     0
215     1
219     0
230     0
233     1
234     0
250     0
279     0
282     1
290     1
309     0
325 

Raw Ys = 

 31      1
32      0
38      0
104     0
141     0
146     0
200     0
216     0
225     0
240     0
244     0
252     0
254     0
255     0
265     0
283     0
296     0
303     0
333     0
351     0
366     0
387     0
391     0
394     0
417     0
418     0
419     0
427     0
441     0
478     0
494     0
537     0
544     0
546     0
588     0
595     0
600     0
615     0
625     0
671     0
692     0
703     0
723     0
763     0
766     0
767     0
800     0
808     0
813     0
821     0
856     0
857     0
858     0
863     0
867     0
881     0
885     0
910     0
933     0
934     0
957     0
967     0
980     0
981     0
984     0
1010    0
1026    0
1027    0
1029    0
1045    0
1054    0
1056    0
1069    0
1083    0
1086    0
1119    0
1176    0
1192    0
1196    0
1267    0
1283    0
1293    0
1317    0
1318    0
1327    0
1338    0
1350    0
1352    0
1354    0
1359    0
1362    0
1363    0
1372    0
1389    0
1401    0
1424    0
1428    0
1431    0
1439    

Original X data shape = (137, 127), golden_answer_shape = (137, 1)
We have 2 classes
Classes are =  [0 1]
Accuracy for not unambig is = 0.9565217391304348 


Confusion Matrix:
[[1. 0.]
 [1. 0.]]



Right before training, Y =  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
Raw Ys = 

 20      0
26      0
121     0
126     0
157     0
206     0
218     0
236     0
262     0
274     0
280     0
304     0
320     0
321     1
330     0
334     0
337     0
342     0
386     0
420     1
429     0
452     0
458     0
473     0
480     0
483     0
484     0
514     0
519     0
521     0
524     0
534     0
535     0
553     0
555     0


Original X data shape = (85, 133), golden_answer_shape = (85, 1)
We have 2 classes
Classes are =  [0 1]
Accuracy for not hl is = 1.0 


Confusion Matrix:
[[1.]]






QID = 'make_list_from_zero_to_x.'
Right before training, Y =  [0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0]
Raw Ys = 

 23      0
56      0
57      0
74      1
98      1
138     0
172     0
176     0
246     0
275     0
284     1
285     0
314     0
365     0
369     0
404     1
407     0
440     0
444     0
461     0
474     0
576     0
583     0
607     0
758     0
762     0
825     0
840     0
908     0
1003    0
1014    0
1021    0
1050    0
1063    0
1096    0
1116    1
1179    0
1193    1
1247    0
1268    0
1300    0
1420    0
1437    0
1451    1
1519    0
1525    0
1528    0
1604    0
1651    0
1709    0
1846    0
1927    0
1936

Original X data shape = (45, 46), golden_answer_shape = (45, 1)
We have 2 classes
Classes are =  [0 1]
Accuracy for not unambig is = 0.9375 


Confusion Matrix:
[[1. 0.]
 [1. 0.]]



Right before training, Y =  [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
Raw Ys = 

 181     0
182     0
186     0
187     0
188     1
1777    0
1779    0
1780    0
1781    0
1782    0
1786    0
1787    1
1788    1
1790    0
1792    1
1793    0
1795    0
1796    0
1797    0
1798    1
1802    0
1804    0
1805    0
1806    0
1809    1
1810    1
1881    0
1882    0
2413    0
2414    0
2416    0
2418    0
2419    0
2420    0
2421    1
2507    0
2509    0
2510    0
3045    0
3047    0
3048    0
3050    0
3051    1
3052    0
3054    0
------------------------
validating on not c col
Original X data shape = (45, 46), golden_answer_shape = (45, 1)
We have 2 classes
Classes are =  [0 1]
Accuracy for not c is = 0.875 


Confus

In [296]:
#one_v_all - list of 3d lists
print("Hi there")

one_v_all_accuracy = np.array(one_v_all)
print("Averaging results for all qids in each of the respective dimensions\n")

avg_unambig_accuracy_one_v_all = np.mean(one_v_all_accuracy[:, 0])
print("Avg accuracy for Unambig: {:.2f}".format(avg_unambig_accuracy))


avg_corr_accuracy_one_v_all_accuracy = np.mean(one_v_all_accuracy[:, 1])
print("Avg accuracy for Correct column: {:.2f}".format(avg_corr_accuracy_one_v_all_accuracy))



avg_highlvl_accuracy_one_v_all_accuracy = np.mean(one_v_all_accuracy[:, 2])
print("Avg accuracy for High-level column: {:.2f}\n\n".format(avg_highlvl_accuracy_one_v_all_accuracy))


avg_classifier_perf_overall_one_v_all_accuracy = np.mean(one_v_all_accuracy)
print("Average results across all dimensions for all qids: {:.2f}".format(avg_classifier_perf_overall_one_v_all_accuracy))

print("Hello there")

Hi there
Averaging results for all qids in each of the respective dimensions

Avg accuracy for Unambig: 0.62
Avg accuracy for Correct column: 0.81
Avg accuracy for High-level column: 0.96


Average results across all dimensions for all qids: 0.89
Hello there


In [297]:

def train_model_and_validate_specify_order(given_train_df, given_validate_df, golden_answers = [], given_dims_order = [], qname = ""):
    
    ydims = given_dims_order #["una", "c", "hl"]
    

    X_train_processed_text = process_raw_features(given_train_df, pct_for_train = 1)
 
    vectorizer = CountVectorizer(ngram_range = (2,2), min_df = 2)
    vec_fitter = vectorizer.fit(X_train_processed_text)    #fit only on training data to prevent overfit
    
    
    X_train = vec_fitter.transform(X_train_processed_text).toarray() #transform data into vectors and convert it to array    
    
    
    
    y_train = given_train_df[ydims].to_numpy(dtype=int)
    
    X_validate_processed_text = process_raw_features(given_validate_df, pct_for_train = 1)
    X_validate = vec_fitter.transform(X_validate_processed_text).toarray()  #use the previously fitted vectorizer to maintain shape
    y_validate = given_validate_df[ydims].to_numpy(dtype=int)
    
    if len(golden_answers) > 0:
    
        distance_from_golden_vector_trainset = add_similarity_vector(golden_answers, X_train, vec_fitter)
        distance_from_golden_vector_validationset  = add_similarity_vector(golden_answers, X_validate, vec_fitter)
        
        print("Original X data shape = {}, golden_answer_shape = {}".format(X_train.shape,
                                                                           distance_from_golden_vector_trainset.shape))
        
        X_train = np.append(X_train, distance_from_golden_vector_trainset, axis = 1)
        X_validate = np.append(X_validate, distance_from_golden_vector_validationset, axis = 1)
        
#         X_train.append(distance_from_golden_vector_trainset, axis = 1)
#         X_validate.append(distance_from_golden_vector_validationset, axis = 1)
    
    
    
        
    labels_order = ["Unambig", "Correct", "High level"]
    accuracy_l = []
  
    
    col1name, col2name, col3name = given_dims_order #order of the y dimensions
    
    y_firstcol_train_vector = given_train_df[col1name].to_numpy(dtype=int) 
    y_secondcol_train_vector = given_train_df[col2name].to_numpy(dtype=int)
    y_thirdcol_train_vector = given_train_df[col3name].to_numpy(dtype=int)
    
    
    #potential
    
    firstdim_feedback_indices = []
    seconddim_feedback_indices = []
    thirddim_feedback_indices = []
    
    
    num_classes = list(np.unique(y_firstcol_train_vector)) #check number of classes in data. should be two
    if len(num_classes) == 1: #change the last y in the tuple to force training to go through - otherwise, throwing error
        y_firstcol_train_vector[-1] = 0 if y_firstcol_train_vector[-1] == 1 else 0

        
    model1 = LogisticRegression(random_state = 0)
    model1 = model1.fit(X_train, y_firstcol_train_vector) #train only one column at a time
    round1_y_predictions = model1.predict(X_train)
    
    
    for i in range(len(round1_y_predictions)):
        if round1_y_predictions[i] == 1: #no feedback needed for that dimension. feedback potentially for next dimension
            firstdim_feedback_indices.append(i)

    round2_X_train_input = []
    round2_y_train_input = []
    for cur_index in firstdim_feedback_indices:
        round2_X_train_input.append(X_train[cur_index, :])
        round2_y_train_input.append(y_secondcol_train_vector[cur_index])
    
    
    round2_X_train_input = np.array(round2_X_train_input)
    round2_y_train_input = np.array(round2_y_train_input)
    
        
    
    num_classes2 = list(np.unique(round2_y_train_input)) #check number of classes in data. should be two
    if len(num_classes2) == 1: #change the last y in the tuple to force training to go through - otherwise, throwing error
        round2_y_train_input[-1] = 0 if round2_y_train_input[-1] == 1 else 0

        
        
        
    model2 = LogisticRegression(random_state = 0)
    model2 = model2.fit(round2_X_train_input, round2_y_train_input)
    round2_y_predictions = model2.predict(round2_X_train_input)
    
    
    round3_X_train_input = []
    round3_y_train_input = []
    for i  in range(len(round2_y_predictions)):
        
        if round2_y_predictions[i] == 1: #no feedback needed for that dimension. feedback potentially needed for next dinmension
            
            round3_X_train_input.append(round2_X_train_input[i])
            round3_y_train_input.append(y_thirdcol_train_vector[i])
    
        
    num_classes3 = list(np.unique(round3_y_train_input)) #check number of classes in data. should be two
    if len(num_classes3) == 1: #change the last y in the tuple to force training to go through - otherwise, throwing error
        round3_y_train_input[-1] = 0 if round3_y_train_input[-1] == 1 else 0

        
    model3 = LogisticRegression(random_state = 0)
    model3 = model3.fit(round3_X_train_input, round3_y_train_input)
    round3_y_predictions = model3.predict(round3_X_train_input)
    
    classification_predictions = []
    
    print("input validation =", type(X_validate), " shape", X_validate.shape)
    classification_predictions = model1.predict(X_validate).reshape(-1,1)
    #print("model 1 predictions type = ", type(cola))
    classification_predictions = np.append(classification_predictions, model2.predict(X_validate).reshape(-1,1), axis = 1)
    classification_predictions = np.append(classification_predictions, model3.predict(X_validate).reshape(-1,1), axis = 1)
    
    
    
#     for given_response_vec in X_validate:
#         print("asking for prediction for = ", type(given_response_vec))
#         classifed = model1.predict(given_response_vec) + model2.predict(given_response_vec) + model3.predict(given_response_vec)
#         print("prediction should be list = ", type(classified))
#         classification_predictions.append(classified)
    
   #classification_predictions = np.array(classification_predictions)
    
    accuracy_l = []
    for i in range(3):
        dim_accuracy = metrics.accuracy_score(y_validate[:, i], classification_predictions[:,i])
        print("Accuracy is = ", dim_accuracy)
        accuracy_l.append(dim_accuracy)
        
        print("\n\nConfusion Matrix:")
        
        confusion_matrix = metrics.confusion_matrix(y_validate[:, i], classification_predictions[:,i], normalize="true")
        print(confusion_matrix)
        
        #uncomment next line to print the confusion matrix
        #plot_confusion_matrix(confusion_matrix, labels_order[i] + qname,  dim_accuracy)
        print("\n\n")
        
    return accuracy_l  #accuracy for the three dimensions for a given problem

In [298]:

def train_model_and_validate_specify_order(given_train_df, given_test_df, golden_answers = [], given_dims_order = [], qname = ""):
    
    ydims = given_dims_order #["una", "c", "hl"]
    

    X_train_processed_text = process_raw_features(given_train_df, pct_for_train = 1)
 
    vectorizer = CountVectorizer(ngram_range = (2,2), min_df = 2)
    vec_fitter = vectorizer.fit(X_train_processed_text)    #fit only on training data to prevent overfit
    
    
    X_train = vec_fitter.transform(X_train_processed_text).toarray() #transform data into vectors and convert it to array    
    y_train = given_train_df[ydims].to_numpy(dtype=int)
    
   
    
    
    X_test_processed_text = process_raw_features(given_test_df, pct_for_train = 1)
    X_test = vec_fitter.transform(X_test_processed_text).toarray()
    y_test = given_test_df[ydims].to_numpy(dtype=int)
    
    
    if len(golden_answers) > 0:
    
        distance_from_golden_vector_trainset = add_similarity_vector(golden_answers, X_train, vec_fitter)
        distance_from_golden_vector_testset  = add_similarity_vector(golden_answers, X_test, vec_fitter)
        
        print("Original X data shape = {}, golden_answer_shape = {}".format(X_train.shape,
                                                                           distance_from_golden_vector_trainset.shape))
        
        X_train = np.append(X_train, distance_from_golden_vector_trainset, axis = 1)
        X_test = np.append(X_test, distance_from_golden_vector_testset, axis = 1)
        
#         X_train.append(distance_from_golden_vector_trainset, axis = 1)
#         X_validate.append(distance_from_golden_vector_validationset, axis = 1)
    
    
    
        
    labels_order = ["Unambig", "Correct", "High level"]
    accuracy_l = []
  
    
    col1name, col2name, col3name = given_dims_order #order of the y dimensions
    
    y_firstcol_train_vector = given_train_df[col1name].to_numpy(dtype=int) 
    y_secondcol_train_vector = given_train_df[col2name].to_numpy(dtype=int)
    y_thirdcol_train_vector = given_train_df[col3name].to_numpy(dtype=int)
    
    
    #potential
    
    firstdim_feedback_indices = []
    seconddim_feedback_indices = []
    thirddim_feedback_indices = []
    
    
    num_classes = list(np.unique(y_firstcol_train_vector)) #check number of classes in data. should be two
    if len(num_classes) == 1: #change the last y in the tuple to force training to go through - otherwise, throwing error
        y_firstcol_train_vector[-1] = 0 if y_firstcol_train_vector[-1] == 1 else 0

        
    model1 = LogisticRegression(random_state = 0)
    model1 = model1.fit(X_train, y_firstcol_train_vector) #train only one column at a time
    round1_y_predictions = model1.predict(X_train)
    
    
    for i in range(len(round1_y_predictions)):
        if round1_y_predictions[i] == 1: #no feedback needed for that dimension. feedback potentially for next dimension
            firstdim_feedback_indices.append(i)

    round2_X_train_input = []
    round2_y_train_input = []
    for cur_index in firstdim_feedback_indices:
        round2_X_train_input.append(X_train[cur_index, :])
        round2_y_train_input.append(y_secondcol_train_vector[cur_index])
    
    
    round2_X_train_input = np.array(round2_X_train_input)
    round2_y_train_input = np.array(round2_y_train_input)
    
        
    
    num_classes2 = list(np.unique(round2_y_train_input)) #check number of classes in data. should be two
    if len(num_classes2) == 1: #change the last y in the tuple to force training to go through - otherwise, throwing error
        round2_y_train_input[-1] = 0 if round2_y_train_input[-1] == 1 else 0

        
        
        
    model2 = LogisticRegression(random_state = 0)
    model2 = model2.fit(round2_X_train_input, round2_y_train_input)
    round2_y_predictions = model2.predict(round2_X_train_input)
    
    
    round3_X_train_input = []
    round3_y_train_input = []
    for i  in range(len(round2_y_predictions)):
        
        if round2_y_predictions[i] == 1: #no feedback needed for that dimension. feedback potentially needed for next dinmension
            
            round3_X_train_input.append(round2_X_train_input[i])
            round3_y_train_input.append(y_thirdcol_train_vector[i])
    
        
    num_classes3 = list(np.unique(round3_y_train_input)) #check number of classes in data. should be two
    if len(num_classes3) == 1: #change the last y in the tuple to force training to go through - otherwise, throwing error
        round3_y_train_input[-1] = 0 if round3_y_train_input[-1] == 1 else 0

        
    model3 = LogisticRegression(random_state = 0)
    model3 = model3.fit(round3_X_train_input, round3_y_train_input)
    round3_y_predictions = model3.predict(round3_X_train_input)
    
    classification_predictions = []
    
    print("input validation =", type(X_test), " shape", X_test.shape)
    classification_predictions = model1.predict(X_test).reshape(-1,1)
    #print("model 1 predictions type = ", type(cola))
    classification_predictions = np.append(classification_predictions, model2.predict(X_test).reshape(-1,1), axis = 1)
    classification_predictions = np.append(classification_predictions, model3.predict(X_test).reshape(-1,1), axis = 1)
    
    
    
#     for given_response_vec in X_validate:
#         print("asking for prediction for = ", type(given_response_vec))
#         classifed = model1.predict(given_response_vec) + model2.predict(given_response_vec) + model3.predict(given_response_vec)
#         print("prediction should be list = ", type(classified))
#         classification_predictions.append(classified)
    
   #classification_predictions = np.array(classification_predictions)
    
    accuracy_l = []
    for i in range(3):
        dim_accuracy = metrics.accuracy_score(y_test[:, i], classification_predictions[:,i])
        print("Accuracy on test set is = ", dim_accuracy)
        accuracy_l.append(dim_accuracy)
        
        print("\n\nConfusion Matrix:")
        
        confusion_matrix = metrics.confusion_matrix(y_test[:, i], classification_predictions[:,i], normalize="true")
        print(confusion_matrix)
        
        
        
        
        #uncomment next line to print the confusion matrix
        #plot_confusion_matrix(confusion_matrix, labels_order[i] + qname,  dim_accuracy)
        print("\n\n")
    
    print(metrics.classification_report(y_test, classification_predictions))
    return accuracy_l  #accuracy for the three dimensions for a given problem

In [299]:
#for ast

def train_model_and_validate_specify_order(given_train_df, given_test_df, golden_answers = [], given_dims_order = [], qname = ""):
    
    ydims = given_dims_order #["una", "c", "hl"]
    

    X_train_processed_text = process_raw_features(given_train_df, pct_for_train = 1)
 
    vectorizer = CountVectorizer(ngram_range = (2,2), min_df = 2)
    vec_fitter = vectorizer.fit(X_train_processed_text)    #fit only on training data to prevent overfit
    
    
    X_train = vec_fitter.transform(X_train_processed_text).toarray() #transform data into vectors and convert it to array    
    y_train = given_train_df[ydims].to_numpy(dtype=int)
    
   
    
    
    X_test_processed_text = process_raw_features(given_test_df, pct_for_train = 1)
    X_test = vec_fitter.transform(X_test_processed_text).toarray()
    y_test = given_test_df[ydims].to_numpy(dtype=int)
    
    
    if len(golden_answers) > 0:
    
        distance_from_golden_vector_trainset = add_similarity_vector(golden_answers, X_train, vec_fitter)
        distance_from_golden_vector_testset  = add_similarity_vector(golden_answers, X_test, vec_fitter)
        
        print("Original X data shape = {}, golden_answer_shape = {}".format(X_train.shape,
                                                                           distance_from_golden_vector_trainset.shape))
        
        X_train = np.append(X_train, distance_from_golden_vector_trainset, axis = 1)
        X_test = np.append(X_test, distance_from_golden_vector_testset, axis = 1)
        
#         X_train.append(distance_from_golden_vector_trainset, axis = 1)
#         X_validate.append(distance_from_golden_vector_validationset, axis = 1)
    
    
    
        
    labels_order = ["Unambig", "Correct", "High level"]
    accuracy_l = []
  
    
    col1name, col2name, col3name = given_dims_order #order of the y dimensions
    
    y_firstcol_train_vector = given_train_df[col1name].to_numpy(dtype=int) 
    y_secondcol_train_vector = given_train_df[col2name].to_numpy(dtype=int)
    y_thirdcol_train_vector = given_train_df[col3name].to_numpy(dtype=int)
    
    
    #potential
    
    firstdim_feedback_indices = []
    seconddim_feedback_indices = []
    thirddim_feedback_indices = []
    
    
    num_classes = list(np.unique(y_firstcol_train_vector)) #check number of classes in data. should be two
    if len(num_classes) == 1: #change the last y in the tuple to force training to go through - otherwise, throwing error
        y_firstcol_train_vector[-1] = 0 if y_firstcol_train_vector[-1] == 1 else 0

        
    model1 = LogisticRegression(random_state = 0)
    model1 = model1.fit(X_train, y_firstcol_train_vector) #train only one column at a time
    round1_y_predictions = model1.predict(X_train)
    
    
    for i in range(len(round1_y_predictions)):
        if round1_y_predictions[i] == 1: #no feedback needed for that dimension. feedback potentially for next dimension
            firstdim_feedback_indices.append(i)

    round2_X_train_input = []
    round2_y_train_input = []
    for cur_index in firstdim_feedback_indices:
        round2_X_train_input.append(X_train[cur_index, :])
        round2_y_train_input.append(y_secondcol_train_vector[cur_index])
    
    
    round2_X_train_input = np.array(round2_X_train_input)
    round2_y_train_input = np.array(round2_y_train_input)
    
        
    
    num_classes2 = list(np.unique(round2_y_train_input)) #check number of classes in data. should be two
    if len(num_classes2) == 1: #change the last y in the tuple to force training to go through - otherwise, throwing error
        round2_y_train_input[-1] = 0 if round2_y_train_input[-1] == 1 else 0

        
        
        
    model2 = LogisticRegression(random_state = 0)
    model2 = model2.fit(round2_X_train_input, round2_y_train_input)
    round2_y_predictions = model2.predict(round2_X_train_input)
    
    
    round3_X_train_input = []
    round3_y_train_input = []
    for i  in range(len(round2_y_predictions)):
        
        if round2_y_predictions[i] == 1: #no feedback needed for that dimension. feedback potentially needed for next dinmension
            
            round3_X_train_input.append(round2_X_train_input[i])
            round3_y_train_input.append(y_thirdcol_train_vector[i])
    
        
    num_classes3 = list(np.unique(round3_y_train_input)) #check number of classes in data. should be two
    if len(num_classes3) == 1: #change the last y in the tuple to force training to go through - otherwise, throwing error
        round3_y_train_input[-1] = 0 if round3_y_train_input[-1] == 1 else 0

        
    model3 = LogisticRegression(random_state = 0)
    model3 = model3.fit(round3_X_train_input, round3_y_train_input)
    round3_y_predictions = model3.predict(round3_X_train_input)
    
    classification_predictions = []
    
    print("input validation =", type(X_test), " shape", X_test.shape)
    classification_predictions = model1.predict(X_test).reshape(-1,1)
    #print("model 1 predictions type = ", type(cola))
    classification_predictions = np.append(classification_predictions, model2.predict(X_test).reshape(-1,1), axis = 1)
    classification_predictions = np.append(classification_predictions, model3.predict(X_test).reshape(-1,1), axis = 1)
    
    
    
#     for given_response_vec in X_validate:
#         print("asking for prediction for = ", type(given_response_vec))
#         classifed = model1.predict(given_response_vec) + model2.predict(given_response_vec) + model3.predict(given_response_vec)
#         print("prediction should be list = ", type(classified))
#         classification_predictions.append(classified)
    
   #classification_predictions = np.array(classification_predictions)
    
    accuracy_l = []
    for i in range(3):
        dim_accuracy = metrics.accuracy_score(y_test[:, i], classification_predictions[:,i])
        print("Accuracy on test set is = ", dim_accuracy)
        accuracy_l.append(dim_accuracy)
        
        print("\n\nConfusion Matrix:")
        
        confusion_matrix = metrics.confusion_matrix(y_test[:, i], classification_predictions[:,i], normalize="true")
        print(confusion_matrix)
        
        
        
        
        #uncomment next line to print the confusion matrix
        #plot_confusion_matrix(confusion_matrix, labels_order[i] + qname,  dim_accuracy)
        print("\n\n")
    
    print(metrics.classification_report(y_test, classification_predictions))
    return accuracy_l  #accuracy for the three dimensions for a given problem

In [300]:
import ast

best_order = None
best_order_acc = 0

for dims_order in [["una", "c", "hl"], ["una", "hl" , "c"], ["hl", "una", "c"], ["hl", "c", "una"], ["c", "hl", "una"], 
                   ["c", "una", "hl"]]:
    
        
        qids_accuracy = []
     
        print("--------------------------------------------------------------------------------------------------\n")
        print("Cur order = {}".format(dims_order))
        for cur_qid in alldata_df["qid"].unique().tolist():


            cur_question_training_df = train_df[train_df["qid"] == cur_qid]
            cur_question_validation_df = validate_df[validate_df["qid"] == cur_qid]

            print("QID = \'{}.\'".format(cur_qid))
            qname_formatted = ": " + cur_qid #format cur_qid name so it looks nice when printed with other function

            q_goldenanswers = cur_question_training_df["example_correct_answers"].iloc[0]
            q_goldenanswers = ast.literal_eval(q_goldenanswers)

            if type(q_goldenanswers) != list:
                print("Had to convert to list")
                q_goldenanswers = [q_goldenanswers]


            cur_qid_accuracy = train_model_and_validate_specify_order(cur_question_training_df, cur_question_validation_df, 
                                                        qname = qname_formatted, golden_answers = q_goldenanswers, 
                                                                      given_dims_order = dims_order)

            qids_accuracy.append(cur_qid_accuracy)
            print("\n\n")


        qids_accuracy = np.array(qids_accuracy)
        print("Averaging results for all qids in each of the respective dimensions\n")

        avg_unambig_accuracy = np.mean(qids_accuracy[:, 0])
        print("Avg accuracy for Unambig: {:.2f}".format(avg_unambig_accuracy))


        avg_corr_accuracy = np.mean(qids_accuracy[:, 1])
        print("Avg accuracy for Correct column: {:.2f}".format(avg_corr_accuracy))



        avg_highlvl_accuracy = np.mean(qids_accuracy[:, 2])
        print("Avg accuracy for High-level column: {:.2f}\n\n".format(avg_highlvl_accuracy))


        avg_classifier_perf_overall = np.mean(qids_accuracy)
        print("Average results across all dimensions for all qids: {:.2f}".format(avg_classifier_perf_overall))
    
        if best_order_acc < avg_classifier_perf_overall:
            best_order = dims_order
            best_order_acc = avg_classifier_perf_overall

--------------------------------------------------------------------------------------------------

Cur order = ['una', 'c', 'hl']
QID = 'print_found_if_three_numbers_unique.'
Original X data shape = (96, 116), golden_answer_shape = (96, 1)
input validation = <class 'numpy.ndarray'>  shape (32, 117)
Accuracy on test set is =  0.5625


Confusion Matrix:
[[0.23529412 0.76470588]
 [0.06666667 0.93333333]]



Accuracy on test set is =  0.78125


Confusion Matrix:
[[0.         1.        ]
 [0.03846154 0.96153846]]



Accuracy on test set is =  0.84375


Confusion Matrix:
[[0. 1.]
 [0. 1.]]



              precision    recall  f1-score   support

           0       0.52      0.93      0.67        15
           1       0.81      0.96      0.88        26
           2       0.84      1.00      0.92        27

   micro avg       0.73      0.97      0.84        68
   macro avg       0.72      0.96      0.82        68
weighted avg       0.76      0.97      0.85        68
 samples avg       0.73  

c:\users\chine\appdata\local\programs\python\python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Original X data shape = (72, 59), golden_answer_shape = (72, 1)
input validation = <class 'numpy.ndarray'>  shape (24, 60)
Accuracy on test set is =  0.9166666666666666


Confusion Matrix:
[[0. 1.]
 [0. 1.]]



Accuracy on test set is =  0.9583333333333334


Confusion Matrix:
[[0. 1.]
 [0. 1.]]



Accuracy on test set is =  0.5833333333333334


Confusion Matrix:
[[0.2        0.8       ]
 [0.14285714 0.85714286]]



              precision    recall  f1-score   support

           0       0.92      1.00      0.96        22
           1       0.96      1.00      0.98        23
           2       0.60      0.86      0.71        14

   micro avg       0.84      0.97      0.90        59
   macro avg       0.83      0.95      0.88        59
weighted avg       0.86      0.97      0.91        59
 samples avg       0.84      0.97      0.88        59




QID = 'does_list_have_value_above_y.'
Original X data shape = (45, 69), golden_answer_shape = (45, 1)
input validation = <class 'numpy.ndarray'

c:\users\chine\appdata\local\programs\python\python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


input validation = <class 'numpy.ndarray'>  shape (34, 101)
Accuracy on test set is =  0.8823529411764706


Confusion Matrix:
[[0.5 0.5]
 [0.  1. ]]



Accuracy on test set is =  0.7352941176470589


Confusion Matrix:
[[0.30769231 0.69230769]
 [0.         1.        ]]



Accuracy on test set is =  0.8529411764705882


Confusion Matrix:
[[0.         1.        ]
 [0.03333333 0.96666667]]



              precision    recall  f1-score   support

           0       0.87      1.00      0.93        26
           1       0.70      1.00      0.82        21
           2       0.88      0.97      0.92        30

   micro avg       0.82      0.99      0.89        77
   macro avg       0.82      0.99      0.89        77
weighted avg       0.83      0.99      0.90        77
 samples avg       0.79      0.90      0.82        77




QID = 'a33_replace_all_ys_with_zs.'


...

c:\users\chine\appdata\local\programs\python\python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Original X data shape = (85, 133), golden_answer_shape = (85, 1)
input validation = <class 'numpy.ndarray'>  shape (28, 134)
Accuracy on test set is =  0.5357142857142857


Confusion Matrix:
[[0.4375     0.5625    ]
 [0.33333333 0.66666667]]



Accuracy on test set is =  0.7857142857142857


Confusion Matrix:
[[0. 1.]
 [0. 1.]]



Accuracy on test set is =  0.9285714285714286


Confusion Matrix:
[[0. 1.]
 [0. 1.]]



              precision    recall  f1-score   support

           0       0.47      0.67      0.55        12
           1       0.79      1.00      0.88        22
           2       0.93      1.00      0.96        26

   micro avg       0.77      0.93      0.84        60
   macro avg       0.73      0.89      0.80        60
weighted avg       0.78      0.93      0.85        60
 samples avg       0.77      0.93      0.83        60




QID = 'make_list_from_zero_to_x.'
Original X data shape = (78, 95), golden_answer_shape = (78, 1)
input validation = <class 'numpy.ndarray'> 

c:\users\chine\appdata\local\programs\python\python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\chine\appdata\local\programs\python\python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


QID = 'absolute_value.'
Original X data shape = (45, 46), golden_answer_shape = (45, 1)
input validation = <class 'numpy.ndarray'>  shape (15, 47)
Accuracy on test set is =  0.7333333333333333


Confusion Matrix:
[[1.         0.        ]
 [0.44444444 0.55555556]]



Accuracy on test set is =  0.4666666666666667


Confusion Matrix:
[[0. 1.]
 [0. 1.]]



Accuracy on test set is =  0.4666666666666667


Confusion Matrix:
[[0.  1. ]
 [0.3 0.7]]



              precision    recall  f1-score   support

           0       1.00      0.56      0.71         9
           1       0.47      1.00      0.64         7
           2       0.58      0.70      0.64        10

   micro avg       0.59      0.73      0.66        26
   macro avg       0.68      0.75      0.66        26
weighted avg       0.70      0.73      0.66        26
 samples avg       0.57      0.60      0.56        26




QID = 'is_even.'
Original X data shape = (41, 39), golden_answer_shape = (41, 1)
input validation = <class 'numpy.n

c:\users\chine\appdata\local\programs\python\python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [301]:
print("Best order is {}  with accuracy of {}".format(best_order, best_order_acc))

Best order is ['c', 'una', 'hl']  with accuracy of 0.7698478389227691


In [302]:
a = np.array([15, 6, 7]).reshape(1,-1)
b = np.array([30, 12, 14]).reshape(1,-1)

cosine_similarity(a,b)

0.9999999999999999

In [303]:
confusion_matrix = metrics.confusion_matrix([1], [2], normalize="true")

In [304]:
print(confusion_matrix)

[[0. 1.]
 [0. 0.]]


In [305]:
print(metrics.classification_report([2,3], [2,4]))

              precision    recall  f1-score   support

           2       1.00      1.00      1.00         1
           3       0.00      0.00      0.00         1
           4       0.00      0.00      0.00         0

    accuracy                           0.50         2
   macro avg       0.33      0.33      0.33         2
weighted avg       0.50      0.50      0.50         2



c:\users\chine\appdata\local\programs\python\python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\chine\appdata\local\programs\python\python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\chine\appdata\local\programs\python\python39\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, 